<a href="https://colab.research.google.com/github/ktxdev/ml_ai/blob/main/optimization_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Optimization algorithms are mathematical techniques used to identify the best solution by minimizing or maximizing an objective function, playing a crucial role in fields such as machine learning and operations research. This exploration focuses on three prominent algorithms: Gradient Descent, which iteratively adjusts solutions in the direction of the steepest descent to find local or global minima; Simulated Annealing, a probabilistic approach inspired by metallurgy that escapes local optima by accepting worse solutions with decreasing probability, making it effective for discrete and combinatorial problems; and Evolutionary Computing (Genetic Algorithms), which evolves a population of solutions through natural selection mechanisms like selection, crossover, and mutation, suited for complex and non-differentiable challenges. Each algorithm has unique strengths and is designed for different optimization problems, and the following sections will detail their working principles, strengths and weaknesses, along with Python implementations.

# Gradient Descent
- Gradient Descent is an iterative optimization algorithm used to minimize a function by moving in the direction of the steepest descent, i.e., the negative of the gradient.
- For a given function $f(x)$, the gradient is the vector of its partial derivatives.
- Gradient Descent takes small steps proportional to the negative of the gradient until it converges to a local or global minimum.
- **Types of Gradient Descent:**
  1. **Batch Gradient Descent:** Computes the gradient using the whole dataset in each iteration.
  2. **Stochastic Gradient Descent (SGD):** Uses one random sample to compute the gradient.
  3. **Mini-batch Gradient Descent:** Uses a small subset (mini-batch) of data to compute the gradient.
- **Use cases include:** linaer/logistic regression, neural networks training and convex optimization problems where global minimum exists
- It is best suited for convex optimization problems or problems where derivatives are easy to compute.


In [4]:
import numpy as np

def gradient_descent(gradient: callable, start: float, learn_rate: float, n_iter: int = 50, tolerance: float = 1e-06) -> float:
  """Perform gradient descent

  Parameters:
    gradient (callable): A function that returns the gradient of the objective function
    start (float): The initial starting point for the algorithm
    learn_rate (float): A step size that determines how far we move along the gradient
    n_iter (int): Maximum number of iterations (default: 50)
    tolerance (float): A number that determines the convergence threshold

  Returns:
    float: The minimum found by the algorithm
  """

  # Initialize x to the starting point
  x = start

  # Loop over the number of iterations
  for _ in range(n_iter):
    # Compute change in x
    diff = -learn_rate * gradient(x)

    # If converged break the loop
    if np.all(np.abs(diff) <= tolerance):
      break

    # Update current value of x
    x += diff

  return x

def gradient(x):
  """Defines the gradient of the function f(x) = x^2
  """
  return 2 * x

# Run gradient descent starting at x = 10 with learning rate of 0.1
result = gradient_descent(gradient, start = 10, learn_rate = 0.1)
print(f"Minimum point: {result}")

Minimum point: 0.00014272476927059603


# Simulated Annealing (SA)
- The algorithm begins with an initial solution and iteratively makes small changes.
- At each iteration, the algorithm probabilistically decides whether to accept worse solutions to avoid getting stuck in local minima.
- As iterations progress, the acceptance of worse solutions decreases.
- The key to SA is the temperature parameter, which starts high and gradually decreases.
- Early in the process, the algorithm can accept worse solutions to explore the solution space.
- As the temperature decreases, the algorithm becomes more conservative and focuses on refining the current solution.
- **Use cases include:** Comninational optimization problems, Traveling Salesman Problem and Scheduling problems
- It is best suited for non-convex, discrete, or combinatorial optimization problems where there is a risk of getting stuck in local minima

In [6]:
def simulated_annealing(func: callable, bounds: tuple, temp: float = 1000, cooling_rate: float = 0.95, max_iter: int = 1000) -> tuple:
  """Performs Simulated Annealing algorithm

    Parameters:
      func (callable): The objective function to be minimized
      bounds (tuple): The limits for the search space
      temp (float): The initial temperature (default = 1000)
      cooling_rate (float): The rate at which the temperature is decreased (default = 0.95)
      max_iter (int): Maximum number of iterations (default = 1000)

    Returns:
      tuple: The best solution found and it's evaluation
  """

  # Initialize current solution, and best solution
  best = current = np.random.uniform(bounds[0], bounds[1])
  # Set current evalution and best evaluation
  best_eval = current_eval = func(current)

  # Loop over the number of maximum iterations
  for i in range(max_iter):
    # Generate a new candidate solution by adding a number between -1 and 1 to current solution
    candidate = current + np.random.uniform(-1, 1)
    candidate_eval = func(candidate)

    # Evaluate the new candidate with the objective function
    if candidate_eval < current_eval or np.random.rand() < np.exp((current_eval - candidate_eval) / temp):
      current, current_eval = candidate, candidate_eval

    # Update best solution if current solution is better than the one found so far
    if current_eval < best_eval:
      best, best_eval = current, current_eval

    # Decrease the temprature by the cooling rate
    temp *= cooling_rate

  return best, best_eval

def objective(x):
  """Defines the objective function f(x) = x^2
  """
  return x ** 2

# Run Simulated annealing on the function f(x) = x^2 with limits between -10 and 10
best, best_eval = simulated_annealing(objective, bounds=(-10, 10))
print(f"Best solution: {best}, Objective value: {best_eval}")

Best solution: 0.0036999266794541796, Objective value: 1.3689457433336832e-05


# Evolutionary Computing (Genetic Algorithm)
- Evolutionary Computing (EC) is inspired by the process of natural selection.
- This class of algorithms, including Genetic Algorithms (GA), solves optimization problems by evolving a population of candidate solutions over time.
- The process involves:
  - **Selection:** Choosing individuals based on fitness.
  - **Crossover (Recombination):** Combining parts of two individuals to produce offspring.
  - **Mutation:** Introducing random changes to individuals to maintain diversity.
- Each iteration, known as a generation, involves evaluating the fitness of individuals and selecting the best to form the next generation.
- **Use cases include:** Optimization problems with multiple objectives, Feature selection in machine learning and Game playing strategies
- It is best suited for problems where the solution space is large, complex, and non-differentiable, and for problems that require global optimization.

In [18]:
def genetic_algorithm(func: callable, bounds: tuple, pop_size: int = 100, n_generations: int = 100, mutation_rate: float = 0.1):
  """Performs the genetic algorithm

  Parameters:
    func (callable): The objective function to minimize
    bounds (tuple): The bounds for the search space
    pop_size (int): size of the population
    n_generation (int): number of generations for evolution (default = 100)
    mutation_rate (float): Probability of mutation (default = 0.1)

  Returns:
  """
  # Initialize population with random values between the bounds
  population = np.random.uniform(bounds[0], bounds[1], (pop_size, 1))

  # Loop over number of generations
  for generation in range(n_generations):
    # Evaluate the fitness of each individual by applying the objective function
    # to evry individual in the population
    fitness = np.array([func(ind) for ind in population]).flatten()
    # Select first half of the population as parents for the next generation
    sorted_indices = np.argsort(fitness)  # Indices of sorted fitness
    parents = population[sorted_indices][:pop_size // 2]  # Shape: (pop_size // 2, 1)

    offspring = []
    for _ in range(pop_size//2):
      # Randomly select two parents for each new individual to be created
      parent1, parent2 = parents[np.random.randint(0, len(parents), 2)]
      # Create new child by averaging the values of the two parents
      child = (parent1 + parent2) / 2
      if np.random.rand() < mutation_rate:
          child += np.random.uniform(-1, 1)
      offspring.append(child)

    # Form new population by combining the parents and the offspring
    population = np.vstack((parents, offspring))

  best_individual = population[np.argmin([func(ind) for ind in population])]
  return best_individual

def fitness(x):
  """Defines the fittness function f(x) = x^2
  """
  return x ** 2

best = genetic_algorithm(fitness, bounds=(-10, 10))
print(f"Best individual: {best}")


Best individual: [4.60606805e-42]


# Strengths and Weaknesses
|Algorithm|Strengths|Weaknesses|
|---------|---------|----------|
|Gradient Decent|- Fast convergence on convex problems|- Prone to getting stuck in local minima|
| |- Simple to implement and understand|- Requires differentiability|
| |- Efficient for large-scale linear models|- Learning rate tuning can be difficult|
|Simulated Annealing|- Can escape local minima|- Slow convergence|
| |- Simple and flexible|- Requires careful temperature schedule design|
| |- Can handle noisy and discontinuous functions|- No guarantee of finding the global optimum|
|Evolutionary Computing|- Robust to non-differentiable, noisy, or complex landscapes|- Computationally expensive|
| |- Can handle multiple objectives|- Requires tuning multiple hyperparameters (mutation rate, etc.)|
| |- Global optimization potential|- Can converge prematurely if diversity is lost|